In [1]:
import pandas as pd

In [2]:
rideshare_filepath = 'filepath/cleaned_fhvhv_tripdata_2024-06.parquet'
rideshare_df = pd.read_parquet(rideshare_filepath)

In [3]:
rideshare_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20123226 entries, 0 to 20123225
Data columns (total 7 columns):
 #   Column            Dtype         
---  ------            -----         
 0   pickup_datetime   datetime64[ns]
 1   dropoff_datetime  datetime64[ns]
 2   PULocationID      int32         
 3   DOLocationID      int32         
 4   trip_miles        float64       
 5   trip_time         int64         
 6   trip_duration     float64       
dtypes: datetime64[ns](2), float64(2), int32(2), int64(1)
memory usage: 921.2 MB


In [4]:
# Adding is_weekend column
rideshare_df['is_weekend'] = rideshare_df['pickup_datetime'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

# Dropping dropoffs, as not needed
rideshare_df.drop('dropoff_datetime', axis=1, inplace=True)

In [5]:
# Create new columns for pickup date and hour
rideshare_df['pickup_date'] = rideshare_df['pickup_datetime'].dt.date
rideshare_df['pickup_hour'] = rideshare_df['pickup_datetime'].dt.hour

# Group by date, hour, and is_weekend, and count the number of rides
hourly_aggregated_df = rideshare_df.groupby(['pickup_date', 'pickup_hour', 'is_weekend']).size().reset_index(name='ride_count')

hourly_aggregated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pickup_date  720 non-null    object
 1   pickup_hour  720 non-null    int64 
 2   is_weekend   720 non-null    int64 
 3   ride_count   720 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 22.6+ KB


In [24]:
weather_filepath = "filepath/weather_data_june_2024_cleaned.csv"
weather_data = pd.read_csv(weather_filepath)

In [25]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   datetime          720 non-null    object 
 1   Temperature (F)   720 non-null    int64  
 2   Wind Speed (mph)  720 non-null    int64  
 3   Precip. (in)      720 non-null    float64
 4   Condition         720 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 28.2+ KB


In [26]:
# Convert 'datetime' column to datetime type
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])

# Create dummy variables for the 'Condition' column
weather_data_dummies = pd.get_dummies(weather_data, columns=['Condition'])

In [27]:
weather_data_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   datetime                           720 non-null    datetime64[ns]
 1   Temperature (F)                    720 non-null    int64         
 2   Wind Speed (mph)                   720 non-null    int64         
 3   Precip. (in)                       720 non-null    float64       
 4   Condition_Cloudy                   720 non-null    uint8         
 5   Condition_Cloudy / Windy           720 non-null    uint8         
 6   Condition_Fair                     720 non-null    uint8         
 7   Condition_Fair / Windy             720 non-null    uint8         
 8   Condition_Haze                     720 non-null    uint8         
 9   Condition_Light Rain               720 non-null    uint8         
 10  Condition_Light Rain with Thunder  720

In [28]:
# Define the new condition categories and their respective original columns
condition_categories = {
    'Condition_All_Cloudy': [
        'Condition_Cloudy', 'Condition_Cloudy / Windy', 'Condition_Mostly Cloudy',
        'Condition_Mostly Cloudy / Windy', 'Condition_Partly Cloudy', 'Condition_Partly Cloudy / Windy'
    ],
    'Condition_All_Fair': ['Condition_Fair', 'Condition_Fair / Windy'],
    'Condition_All_Fog': ['Condition_Fog', 'Condition_Haze', 'Condition_Haze / Windy', 
                      'Condition_Smoke', 'Condition_Smoke / Windy', 'Condition_Mist'],
    'Condition_All_Rain': ['Condition_Light Drizzle', 'Condition_Light Rain', 'Condition_Light Rain with Thunder'],
    'Condition_All_Thunder': ['Condition_T-Storm', 'Condition_Heavy T-Storm', 'Condition_Thunder', 
                        'Condition_Thunder in the Vicinity'],
}

# Loop through each category and merge columns dynamically
for new_col, old_cols in condition_categories.items():
    # Filter only the columns that exist in the DataFrame
    existing_cols = weather_data_dummies.filter(items=old_cols).columns
    # Sum the existing columns to create the new column
    weather_data_dummies[new_col] = weather_data_dummies[existing_cols].sum(axis=1)

# Drop the original condition columns after merging
columns_to_drop = [col for cols in condition_categories.values() for col in cols if col in weather_data_dummies.columns]
weather_data_dummies.drop(columns=columns_to_drop, inplace=True)

In [29]:
weather_data_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   datetime               720 non-null    datetime64[ns]
 1   Temperature (F)        720 non-null    int64         
 2   Wind Speed (mph)       720 non-null    int64         
 3   Precip. (in)           720 non-null    float64       
 4   Condition_All_Cloudy   720 non-null    int64         
 5   Condition_All_Fair     720 non-null    int64         
 6   Condition_All_Fog      720 non-null    int64         
 7   Condition_All_Rain     720 non-null    int64         
 8   Condition_All_Thunder  720 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(7)
memory usage: 50.8 KB


In [30]:
# Extract date and hour from the datetime column in weather_data_dummies
weather_data_dummies['pickup_date'] = weather_data_dummies['datetime'].dt.date
weather_data_dummies['pickup_hour'] = weather_data_dummies['datetime'].dt.hour

In [31]:
# Merge the two dataframes on pickup_date and pickup_hour
merged_df = pd.merge(hourly_aggregated_df, weather_data_dummies, on=['pickup_date', 'pickup_hour'], how='left')

# Drop the datetime column as it's no longer needed after the merge
merged_df.drop(columns=['datetime'], inplace=True)

In [34]:
# Final check to make sure merge is correct
merged_df.head()

,pickup_date,pickup_hour,is_weekend,ride_count,Temperature (F),Wind Speed (mph),Precip. (in),Condition_All_Cloudy,Condition_All_Fair,Condition_All_Fog,Condition_All_Rain,Condition_All_Thunder
0,2024-06-01,0,1,38854,67,10,0.0,0,1,0,0,0
1,2024-06-01,1,1,30675,67,10,0.0,0,1,0,0,0
2,2024-06-01,2,1,22535,66,9,0.0,0,1,0,0,0
3,2024-06-01,3,1,17082,65,3,0.0,0,1,0,0,0
4,2024-06-01,4,1,13947,64,12,0.0,0,1,0,0,0


In [ ]:
export_filepath = 'filepath/aggregated_fhvhv_tripdata_2023-06.csv'
merged_df.to_csv(export_filepath)